# **GC1**

Il modello GC1 ha la seguente struttura:

#### conv1: First convolutional layer (dim -> 256x256x8)
#### pool: First max pooling layer (dim -> 128x128x8)
#### conv2: Second convolutional layer (dim -> 128x128x16)
#### pool2: Second max pooling layer (dim -> 64x64x16)
#### conv3: Third convolutional layer (dim -> 64x64x32)
#### pool3: Third max pooling layer (dim -> 32x32x32)
#### fc1: Fully connected layer (dim -> num_classes)
#### softmax: Softmax layer (dim -> num_classes)

### **Problemi**

Overfitting: sale sul train ma sul validation si ferma a circa 65%

### **Possibili soluzioni**

- Aggiungere Dropout tra i layer completamente connessi
- Utilizzare tecniche di regularization come L2 regularization
- Sperimentare con diverse architetture di rete, come l'aggiunta di più layer convoluzionali o l'uso di architetture pre-addestrate (transfer learning)

# **GC2**

Il modello GC2 ha la seguente struttura:

#### conv1: First convolutional layer (dim -> 256x256x12)
#### conv2: Second convolutional layer (stride=2, kernel=5) (dim -> 126x126x16)
#### pool: First max pooling layer (dim -> 63x63x16)
#### conv3: Third convolutional layer (dim -> 63x63x24)
#### conv4: Fourth convolutional layer (stride=2, kernel=5) (dim -> 30x30x32)
#### pool2: Second max pooling layer (dim -> 15x15x32)
#### fc1: Fully connected layer (dim -> 1024)
#### dropout: Dropout layer (p=0.2)
#### fc2: Output layer (dim -> num_classes)

### **Modifiche rispetto a GC1**
- Aumentato il numero di filtri iniziali (8→12) e aggiunto un quarto layer convoluzionale per maggiore capacità espressiva.
- Riduzione della risoluzione tramite convoluzioni con stride=2 invece di soli max pooling, così da imparare anche la downsampling function.
- Inserito layer fully-connected intermedio a 1024 neuroni prima dell'output finale (GC1 andava direttamente a num_classes).
- Aggiunto dropout (p=0.2) tra i fully-connected per mitigare l'overfitting.
- Output ora su fc2 dedicato invece di un unico fc finale.

### **Problemi**
- Train accuracy: 100%
- Validation accuracy: 71.56%

### **Soluzioni**

Per ridurre l'eccessivo overfitting e migliorare la generalizzazione del modello GC2, si potrebbero adottare le seguenti strategie:
- **Aumentare il dropout**: Incrementare il tasso di dropout (ad esempio a 0.5) per ridurre ulteriormente la dipendenza del modello da specifici neuroni durante l'addestramento.
- **Batch normalization**: Integrare layer di batch normalization dopo le convoluzioni per stabilizzare e accelerare l'addestramento.
- **Kernel 3x3**: Sostituire i kernel 5x5 con kernel 3x3 per ridurre il numero di parametri e migliorare l'efficienza computazionale grazie allo stesso receptive field.
- **Aumento della Profondità Convoluzionale**: Piuttosto che layer lineari enormi, più layer convoluzionali per estrarre feature di alto livello.
- **Cambiare regolarizzazione**: Rimuovere la l1 regularization e sfruttare solo l2 dell'elastic net, per non perdere informazioni importanti portando a zero i pesi.

# **GC3**

Il modello GC3 ha la seguente struttura:

#### conv1: First convolutional layer + batch normalization + ReLU (dim -> 256x256x16)
#### conv2: Second convolutional layer (stride=2) + batch normalization + ReLU (dim -> 128x128x32)
#### conv3: Third convolutional layer (stride=2) + batch normalization + ReLU (dim -> 64x64x64)
#### pool: Max pooling layer (dim -> 32x32x64)
#### conv4: Fourth convolutional layer (stride=2) + batch normalization + ReLU (dim -> 16x16x128)
#### conv5: Fifth convolutional layer (stride=2) + batch normalization + ReLU (dim -> 8x8x128)
#### conv6: Sixth convolutional layer (stride=2) + batch normalization + ReLU (dim -> 4x4x256)
#### fc1: Fully connected layer (dim -> 1024)
#### dropout: Dropout layer (p=0.5)
#### fc2: Output layer (dim -> num_classes)

### **Modifiche rispetto a GC2**
- Profondità aumentata (6 convolutional block stride-based) con mappe 16→32→64→128→128→256 per una rappresentazione più ricca.
- Batch normalization dopo ogni convoluzione per stabilizzare l'ottimizzazione e ridurre la covariate shift.
- Downsampling quasi interamente tramite stride nelle convoluzioni invece che con max pooling ripetuti, così da imparare la funzione di riduzione.
- Dropout innalzato a 0.5 sul fully connected da 1024 neuroni per contenere l'overfitting.
- Regolarizzazione elastica semplificata: l1 disattivata (l1_alpha=0) e mantenuta solo l2 (0.0008).

### **Risultati**
- Accuratezza di validazione massima: **78.99%** all'epoca 35 (checkpoint: [Results/GC3/garbage_custom_3_best_78.99_35.pth](Results/GC3/garbage_custom_3_best_78.99_35.pth)).
- +7.4 punti circa rispetto al best di GC2 (71.56%).
- La curva di validation è più stabile ma resta un divario con il training, segno di overfitting residuo.

### **Problemi**
- Overfitting ancora presente: la curva di training resta sopra quella di validation.
- Val accuracy si assesta sotto l'80%, suggerendo margine di generalizzazione non sfruttato.

### **Possibili miglioramenti**
- Aumentare l'augmentazione (color jitter, cutout/mixup) per spingere la generalizzazione oltre il 79%.
- Introdurre label smoothing (ε≈0.05-0.1) e un leggero weight decay per regolarizzare ulteriormente.
- Testare scheduler più dolci (cosine annealing + warmup) per evitare plateau precoce della val_loss.
- Sperimentare skip/residual connection leggere o ridurre gli stride iniziali per non perdere dettagli di texture.

# **GC4**

Il modello GC4 ha la seguente struttura:

#### conv1: 3x3 conv + batch norm + ReLU (dim -> 256x256x32)
#### resblock1: Tre conv 3x3 con batch norm e skip connection (dim -> 256x256x32)
#### conv2: 3x3 conv stride=3 + batch norm + ReLU (dim -> 86x86x64)
#### resblock2: Tre conv 3x3 con batch norm e skip connection (dim -> 86x86x64)
#### conv3: 3x3 conv stride=3, padding=2 + batch norm + ReLU (dim -> 30x30x128)
#### resblock3: Tre conv 3x3 con batch norm e skip connection (dim -> 30x30x128)
#### pool: MaxPool 2x2 stride=2 (dim -> 15x15x128)
#### conv4: 3x3 conv stride=2 + batch norm + ReLU (dim -> 8x8x256)
#### conv5: 3x3 conv stride=1 + batch norm + ReLU (dim -> 8x8x512)
#### global_avg_pool: AdaptiveAvgPool2d(1) (dim -> 1x1x512)
#### dropout: p=0.3
#### fc: Fully connected (dim -> num_classes)

### **Modifiche rispetto a GC3**
- Inserite tre residual block (3x3) per canale 32/64/128 con skip identity, così da stabilizzare il deepening e mantenere le feature locali.
- Downsampling più aggressivo con stride=3 nelle prime due fasi (256→86→30) per comprimere rapidamente la risoluzione e ridurre il costo computazionale.
- Canali ampliati fino a 512 prima del classifier e passaggio a global average pooling per eliminare i fully-connected flattenati.
- Dropout moderato (0.3) invece di 0.5 e weight init Kaiming su conv/linear.
- Validazione più rigida sull'input: il forward lancia errori se la shape non è (N, 3, 256, 256).

### **Risultati**
- Accuratezza di validazione massima: **83.26%** all'epoca 50 (checkpoint: [Results/GC4/garbage_custom_4_best_83.26_50.pth](Results/GC4/garbage_custom_4_best_83.26_50.pth)).
- Train accuracy finale intorno al 95-96%, con un gap più contenuto rispetto a GC3.
- La validation accuracy cresce in modo costante fino a ~83% e si appiattisce dopo l'epoca ~45.
- Val loss: ~11→~0.7; Train loss: ~6.5→~1.1 (curve stabili dopo metà training).
- Guadagno di circa +4.3 punti rispetto al best di GC3 (78.99%).

### **Problemi**
- Overfitting ancora presente: il train resta ~10-12 punti sopra la validation.
- Downsampling con stride=3 nelle prime fasi rischia di perdere texture sottili (vetro/plastica).
- Curva di validation si stabilizza presto: il scheduler corrente non spinge oltre l'83% pur con loss in calo lenta.
- Troppi layer convoluzionali e batch normalization potrebbero aver aumentato la complessità senza migliorare la generalizzazione.

### **Possibili miglioramenti**
- Aumentare leggeremente il dropout (0.4) o sperimentare stochastic depth sui residual block per regolarizzare.
- Provare a cambiare la regolarizzazione (weight decay)
- Ridurre il numero di layer della rete per diminuire l'overfitting.
- Sperimentare con scheduler più aggressivi


# **GC5**

Il modello GC5 ha la seguente struttura:

#### conv1: 3x3 conv (in=3, out=4) + ReLU (dim -> 256x256x4)
#### pool1: AvgPool2d 2x2 stride=2 (dim -> 128x128x4)
#### resblock1: Residual block con canali 4→12 e identity adattata (dim -> 128x128x12)
#### pool2: AvgPool2d 2x2 stride=2 (dim -> 64x64x12)
#### resblock2: Residual block con canali 12→36 (dim -> 64x64x36)
#### pool3: AvgPool2d 2x2 stride=2 (dim -> 32x32x36)
#### resblock3: Residual block con canali 36→108 (dim -> 32x32x108)
#### pool4: AvgPool2d 2x2 stride=2 (dim -> 16x16x108)
#### resblock4: Residual block con canali 108→324 (dim -> 16x16x324)
#### pool5: AvgPool2d 2x2 stride=2 (dim -> 8x8x324)
#### conv2: 4x4 conv stride=2 pad=1 (dim -> 4x4x512) + ReLU
#### global_avg_pool: AdaptiveAvgPool2d(1) (dim -> 1x1x512)
#### dropout: p=0.5
#### fc: Fully connected (dim -> num_classes)
#### shape check: il forward valida (N, 3, 256, 256) e solleva errori se diverso

### **Modifiche rispetto a GC4**
- Stem molto più stretto (4 canali iniziali) e downsampling via average pooling: meno parametri e inferenza più leggera.
- Residual block con identity adattata che ampliano progressivamente i canali (4→12→36→108→324) mantenendo stabilità in training.
- Head compatta: una sola conv finale a 512 canali + global average pooling + dropout 0.5, senza fully-connected estesi.
- Input validation esplicita nel forward per evitare errori di shape.
- Regolarizzazione più decisa lato weight decay (l2_alpha=0.0012) e learning rate ridotto (3e-4).

### **Risultati**
- Accuratezza di validazione massima: **85.61%** intorno all'epoca 45 (checkpoint: [Results/GC5/garbage_custom_1_best_85.61_45.pth](Results/GC5/garbage_custom_1_best_85.61_45.pth)).
- Curva di validation accuracy sale rapidamente a ~83% (epoca ~20) e si stabilizza verso 85% con oscillazioni minime; train accuracy ≈95%.
- Loss in calo regolare: train da ~6.2→~1.0, validation da ~1.1→~0.45, senza divergenze nella fase finale.
- +2.35 punti circa rispetto al best di GC4 (83.26%) con modello più leggero e tempi di training/inferenza ridotti.

### **Analisi finale**
- GC5 è il modello finale: migliore accuratezza validazione (+2.35 pt su GC4) e architettura più leggera grazie allo stem ridotto e al GAP senza fully-connected pesanti.
- Il gap train/val è contenuto (≈10 pt) e le curve sono stabili: rischio di overfitting residuo ma sotto controllo.
- Consigliato come default per produzione: input 256x256 RGB, lr 3e-4, l2=0.0012, dropout 0.5.
- Possibili micro-migliorie: augmentazione aggiuntiva su classi minoritarie, label smoothing leggero, valutare quantizzazione post-training per ulteriore velocità.